<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/999_Down_Trendline_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress

def Down_Trend_Line(data, window=5):
    df = data.iloc[:-window].copy()
    hh_pairs = argrelextrema(df['high'].values, comparator=np.greater, order=window)[0]
    highest_point = df.iloc[hh_pairs].nlargest(1, 'high').sort_values(by='datetime')
    max_idx = data.index.get_loc(highest_point.index[0])
    df_next = df.iloc[max_idx:]

    hh_pairs_next = argrelextrema(df_next['high'].values, comparator=np.greater, order=window)[0]
    slopes = []
    intercepts = []

    # Iterate over each next highest point and calculate the slope
    for hh_next in hh_pairs_next:
        next_hp = df_next.iloc[hh_next]
        slope, intercept, _, _, _ = linregress([highest_point.index[0], next_hp.name], [highest_point['high'].values[0], next_hp['high']])
        # Store the slope
        slopes.append(slope)
        intercepts.append(intercept)

    slopes = [slope for slope in slopes if slope != 0]
    intercepts = [intercept for slope, intercept in zip(slopes, intercepts) if slope != 0]
    if slopes:
        # Find the minimum slope and its corresponding intercept
        min_slope_index = np.argmin(np.abs(slopes))
        min_slope = slopes[min_slope_index]
        min_intercept = intercepts[min_slope_index]
        df = data.copy()
        df['trend'] = min_slope * df.index + min_intercept
        df['Entry'] = df['high'] > df['trend']
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)


Titles = ['Hisse Adı', 'Son Fiyat' ,'Trend Değeri','Yüzde', 'Yakınlık Durumu','Kırılma Durumu']
df_down_trend = pd.DataFrame(columns=Titles)
for i in range(0,len(Hisseler)):
    #print(Hisseler[i])
    try:
        order = 5
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=Interval.in_4_hour, n_bars=500)
        data = data.reset_index()
        Trend_line = Down_Trend_Line(data, window=order)
        Entry = False
        Signals = Trend_line.tail(order)
        Signals = Signals.reset_index()
        Last_Price = Signals.loc[order-1, 'high']
        Last_Trend = Signals.loc[order-1, 'trend']
        Last_Perc = ((Signals.loc[order-1,'trend']- Signals.loc[order-1,'high'])/Signals.loc[order-1,'trend'])*100
        Close_Status = False
        Break_Status = False
        if Last_Perc<=5 and Last_Trend>Last_Price:
            Close_Status  = True

        Entry = Signals['Entry'].any() and not Signals['Entry'].all()
        Last_Entry = Signals.loc[order-1,'Entry']
        if Entry==True and Last_Entry==True:
              Break_Status = True

        L1 = [Hisseler[i] ,round(Last_Price,2), round(Last_Trend,2), round(Last_Perc,2),Close_Status,Break_Status]
        print(L1)
        df_down_trend.loc[len(df_down_trend)] = L1
    except:
        pass

df_close = df_down_trend[(df_down_trend['Yakınlık Durumu'] == True)]
df_Breakout = df_down_trend[(df_down_trend['Kırılma Durumu'] == True)]

print('Yakın Olan Hisseler')
print(df_close.to_string())
print('Kırılma Gerçekleşmiş Olanlar')
print(df_Breakout.to_string())

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-x4d0a5nx
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-x4d0a5nx
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


['A1CAP', 27.72, 33.53, 17.32, False, False]
['ACSEL', 144.2, 171.64, 15.99, False, False]
['ADEL', 542.5, 783.5, 30.76, False, False]
['ADESE', 2.24, 2.48, 9.54, False, False]
['ADGYO', 30.6, 28.93, -5.76, False, False]
['AFYON', 12.95, 15.4, 15.92, False, False]
['AGHOL', 350.75, 355.75, 1.41, True, False]
['AGROT', 17.78, 35.02, 49.23, False, False]
['AGYO', 8.21, 9.27, 11.47, False, False]
['AHGAZ', 14.38, 13.22, -8.79, False, False]
['AKBNK', 62.55, 63.06, 0.8, True, False]
['AKCNS', 137.7, 142.48, 3.36, True, False]
['AKENR', 16.06, 18.17, 11.59, False, False]
['AKFGY', 2.31, 2.59, 10.91, False, False]
['AKFYE', 28.58, 26.99, -5.9, False, False]
['AKGRT', 8.23, 8.38, 1.74, True, False]
['AKMGY', 269.25, 369.17, 27.07, False, False]
['AKSA', 123.0, 124.75, 1.4, True, False]
['AKSEN', 44.1, 48.44, 8.95, False, False]
['AKSGY', 17.58, 18.69, 5.94, False, False]
['AKSUE', 12.03, 16.37, 26.52, False, False]
['AKYHO', 8.16, 7.95, -2.59, False, True]
['ALARK', 105.7, 116.66, 9.4, False,

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['PEKGY', 7.73, 28.85, 73.21, False, False]
['PENGD', 8.34, 8.0, -4.22, False, True]
['PENTA', 16.19, 23.5, 31.1, False, False]
['PETKM', 22.06, 20.97, -5.18, False, False]
['PETUN', 88.75, 99.68, 10.97, False, False]
['PINSU', 26.0, 27.08, 3.97, True, False]
['PKART', 114.0, 106.54, -7.0, False, False]
['PKENT', 315.0, 378.41, 16.76, False, False]
['PLTUR', 17.52, 16.93, -3.45, False, False]
['PNLSN', 68.6, 85.88, 20.12, False, False]
['PNSUT', 86.8, 85.21, -1.87, False, True]
['POLHO', 13.55, 13.16, -2.99, False, False]
['POLTK', 8970.0, 18328.93, 51.06, False, False]
['PRDGS', 6.64, 7.43, 10.62, False, False]
['PRKAB', 38.42, 28.83, -33.25, False, False]
['PRKME', 24.76, 24.14, -2.57, False, True]
['PRZMA', 38.18, 63.66, 40.02, False, False]
['PSDTC', 92.2, 92.99, 0.85, True, False]
['PSGYO', 6.08, 6.31, 3.62, True, False]
['QNBFB', 346.25, 404.65, 14.43, False, False]
['QNBFL', 126.9, 131.77, 3.7, True, False]
['QPERP', 5058.0, 473.63, -967.92, False, False]
['QTEMZ', 347.6, 338.68